In [5]:
# !pip install linearmodels # required only once

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# ------------------------------
# Step 1: Load Real Dataset
# ------------------------------
# Load dataset
df = pd.read_csv("FirmLevel_PanelData_10K.csv")

# Convert to categorical
df["FirmID"] = df["FirmID"].astype("category")
df["Year"] = df["Year"].astype("category")

# Create lag variable for dynamic panel model
df["Profitability_Lag1"] = df.groupby("FirmID")["Profitability"].shift(1)
df.dropna(inplace=True)

<ipython-input-3-2cc721cb4f61>:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["Profitability_Lag1"] = df.groupby("FirmID")["Profitability"].shift(1)


In [4]:
# Step 2 for Using linearmodels.RandomEffects
import pandas as pd
from linearmodels.panel import RandomEffects

# Assuming df has 'FirmID' and 'Year' columns
df["Year"] = df["Year"].astype(int)  # Make sure Year is numeric
df = df.set_index(["FirmID", "Year"])

# Define the model (drop missing values if necessary)
exog_vars = ["RND_Expenses", "Advertising_Spends", "Debt_Equity_Ratio", "Firm_Size"]
exog = df[exog_vars]
endog = df["Profitability"]

model_re = RandomEffects(endog, exog)
re_results = model_re.fit()
print(re_results.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:          Profitability   R-squared:                        0.9570
Estimator:              RandomEffects   R-squared (Between):              0.9950
No. Observations:                9000   R-squared (Within):               0.0782
Date:                Thu, May 29 2025   R-squared (Overall):              0.9869
Time:                        19:02:12   Log-likelihood                   -6627.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   5.002e+04
Entities:                        1000   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                  F(4,8996)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):          5.002e+04
                            

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:599: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weighted_sum: DataFrame = frame.groupby(level=level).transform("sum")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:601: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sum_weights: DataFrame = frame.groupby(level=level).transform("sum")
/usr/local/lib/python3.11/dist-packages/linearmodels/panel/data.py:599: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or o

In [7]:
from linearmodels.panel import PanelOLS
import pandas as pd

# Load your dataset
df = pd.read_csv("FirmLevel_PanelData_10K.csv")

# Convert year to integer
df["Year"] = df["Year"].astype(int)

# Set panel structure index (entity and time)
df = df.set_index(["FirmID", "Year"])

# Dependent variable
y = df["Profitability"]

# Independent variables
X = df[["RND_Expenses", "Advertising_Spends", "Debt_Equity_Ratio", "Firm_Size"]]

# Optional: add constant term
from statsmodels.tools.tools import add_constant
X = add_constant(X)

# Estimate the Fixed Effects model
model_fe = PanelOLS(y, X, entity_effects=True)
results_fe = model_fe.fit()
print(results_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:          Profitability   R-squared:                        0.0826
Estimator:                   PanelOLS   R-squared (Between):              0.5471
No. Observations:               10000   R-squared (Within):               0.0826
Date:                Thu, May 29 2025   R-squared (Overall):              0.3301
Time:                        19:07:31   Log-likelihood                   -6834.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      202.49
Entities:                        1000   P-value                           0.0000
Avg Obs:                      10.0000   Distribution:                  F(4,8996)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             202.49
                            

In [9]:
# Refit if needed
fe_res = PanelOLS(y, X, entity_effects=True).fit()
re_res = RandomEffects(y, X).fit()

# Extract parameter estimates
b_fe = fe_res.params
b_re = re_res.params

# Align coefficients for comparison (excluding intercept if needed)
common_coef = b_fe.index.intersection(b_re.index)
diff = b_fe[common_coef] - b_re[common_coef]
cov_diff = fe_res.cov.loc[common_coef, common_coef] - re_res.cov.loc[common_coef, common_coef]

# Hausman test statistic
import numpy as np
from scipy.stats import chi2

hausman_stat = float(diff.T @ np.linalg.inv(cov_diff) @ diff)
df_h = len(diff)
p_value = 1 - chi2.cdf(hausman_stat, df_h)

# Print results
print(f"\n🔍 Hausman Test Results:")
print(f"Chi-Square Statistic: {hausman_stat:.4f}")
print(f"Degrees of Freedom: {df_h}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("✅ Reject null: Fixed Effects preferred (correlation with firm effects exists)")
else:
    print("✅ Fail to reject null: Random Effects is efficient and consistent")


🔍 Hausman Test Results:
Chi-Square Statistic: 2.5307
Degrees of Freedom: 5
P-Value: 0.7719
✅ Fail to reject null: Random Effects is efficient and consistent


In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# For VIF, use non-indexed DataFrame
from statsmodels.api import add_constant
X_vif = add_constant(df.reset_index()[["RND_Expenses", "Advertising_Spends", "Debt_Equity_Ratio", "Firm_Size"]])

print("\n📌 VIF Scores:")
for i, var in enumerate(X_vif.columns):
    vif = variance_inflation_factor(X_vif.values, i)
    print(f"{var}: {vif:.2f}")


📌 VIF Scores:
const: 234.38
RND_Expenses: 1.00
Advertising_Spends: 1.00
Debt_Equity_Ratio: 1.00
Firm_Size: 1.00


In [13]:
from statsmodels.stats.diagnostic import het_white
import statsmodels.api as sm

# Drop constant if previously added, then re-add explicitly
exog = X.drop(columns='const', errors='ignore')
exog_with_const = sm.add_constant(exog)

# Run White test
white_test = het_white(resid, exog_with_const)
labels = ["LM Stat", "LM p-value", "F-stat", "F p-value"]

# Display results
print("\n⚠️ Heteroskedasticity (White Test):")
for name, val in zip(labels, white_test):
    print(f"{name}: {val:.4f}")


⚠️ Heteroskedasticity (White Test):
LM Stat: 20.1174
LM p-value: 0.1265
F-stat: 1.4377
F p-value: 0.1264
